# Class and Static Methods

In [ ]:
# class methods- implemented to act as a alternate constructor and a method to change class variable
from typing import Dict

class Employee:
    """Employee class with class methods for alternate constructors 
    and salary updates."""

    roles: Dict[str, int] = {"VP": 500000, "Manager": 150000, "Intern": 30000}

    def __init__(self, name: str, role: str) -> None:
        self.name = name
        self.role = role
        self.salary = self.roles.get(role, 0)

    @classmethod
    def from_dict(cls, data: Dict[str, str]) -> "Employee":
        """Creates an Employee instance from a dictionary."""
        return cls(data["name"], data["role"])

    @classmethod
    def update_salary(cls, role: str, new_salary: int) -> None:
        """Updates the salary of a given role."""
        if role in cls.roles:
            cls.roles[role] = new_salary
        else:
            print(f"Role '{role}' not found.")


In [3]:

dict1 = {"name": "Hem", "role": "Intern"}
emp1 = Employee.from_dict(dict1)
emp2 = Employee("Lakshman", "Manager")

print(emp1.name, emp1.salary)
print(emp2.name, emp2.salary)

Employee.update_salary("Intern", 50000)

emp3 = Employee("Yarlagadda", "Intern")
print(emp3.name, emp3.salary)


Hem 30000
Lakshman 150000
Yarlagadda 50000


In [4]:

# static methods
from datetime import date

class Person:
    """Person class with a method to calculate age from birth year."""

    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    @classmethod
    def from_birth_year(cls, name: str, year: int) -> "Person":
        """Creates a Person instance using the birth year."""
        return cls(name, date.today().year - year)

    @staticmethod
    def is_adult(age: int) -> bool:
        """Checks if a person is an adult."""
        return age > 18


person1 = Person("Ram", 21)
person2 = Person.from_birth_year("Raj", 2003)

print(person1.age)
print(person2.age)
print(Person.is_adult(14))


21
22
False


In [5]:
#__new__
class PositiveNumber:
    """Ensures a number is positive before creating an instance."""

    def __new__(cls, value: int) -> "PositiveNumber":
        """Creates a new instance only if the value is positive."""
        if value <= 0:
            raise ValueError("Value must be positive!")
        return super().__new__(cls)

    def __init__(self, value: int) -> None:
        """Initializes the PositiveNumber instance."""
        self.value = value


num1 = PositiveNumber(10)
print(num1.value)

try:
    num2 = PositiveNumber(-5)
except ValueError as error:
    print("Error:", error)


10
Error: Value must be positive!


In [6]:
#__new__
class ImmutableConfig:
    """Singleton class to enforce immutable configuration."""

    _instance = None

    def __new__(cls, *args, **kwargs) -> "ImmutableConfig":
        """Creates a single instance and ensures immutability."""
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.__initialized = False
        return cls._instance

    def __init__(self, setting: str) -> None:
        """Initializes the configuration setting only once."""
        if not self.__initialized:
            object.__setattr__(self, "setting", setting)
            object.__setattr__(self, "__initialized", True)

    def __setattr__(self, key: str, value: str) -> None:
        """Prevents modification of attributes after initialization."""
        if hasattr(self, key):
            raise AttributeError("Cannot modify immutable attributes")
        object.__setattr__(self, key, value)


# Example usage
config1 = ImmutableConfig("Dark Mode")
config2 = ImmutableConfig("Light Mode")

print(config1.setting)  # Output: Dark Mode
print(config2.setting)  # Output: Dark Mode (singleton behavior)
print(config1 is config2)  # Output: True (both are same instance)

try:
    config1.setting = "Blue Mode"
except AttributeError as error:
    print("Error:", error)


Light Mode
Light Mode
True
Error: Cannot modify immutable attributes


In [7]:
# __new__
class SingleCharName:
    """Ensures that only a single character is allowed for a name."""

    def __new__(cls, name: str) -> "SingleCharName":
        """Creates a new instance only if the name contains one character."""
        if len(name) != 1:
            raise ValueError("Only one character allowed")
        return super().__new__(cls)

    def __init__(self, name: str) -> None:
        """Initializes the SingleCharName instance with the given character."""
        self.name = name


char1 = SingleCharName("X")
print(char1.name)  

try:
    char2 = SingleCharName("Hem")  # This should raise a ValueError
except ValueError as error:
    print("Error:", error)



X
Error: Only one character allowed


# ITERATORS

In [39]:
mytuple = ("apple", "banana", "cherry")
myit = iter(mytuple)

print(next(myit))
print(next(myit))
print(next(myit))

apple
banana
cherry


In [8]:
#Iterator for default data structures
emp_names = ["A", "B", "C", "D"]
iterator = iter(emp_names)

try:
    while True:
        print(next(iterator))
except StopIteration:
    pass  #Handle Iteration completion





A
B
C
D


In [9]:
#class based iterator
class SalaryIterator:
    """Iterator for calculating updated salaries with a 1.5x increase."""

    def __init__(self, salaries: list[int]) -> None:
        """Initializes the SalaryIterator with a list of salaries."""
        self.salaries = salaries
        self.index = 0

    def __iter__(self) -> "SalaryIterator":
        """Returns the iterator object itself."""
        return self

    def __next__(self) -> str:
        """Calculates and returns the updated salary, stops iteration when complete."""
        if self.index >= len(self.salaries):
            raise StopIteration
        salary = self.salaries[self.index] * 1.50
        self.index += 1
        return f"Updated Salary: {salary:.2f}"


salaries = [50000, 60000, 55000]
salary_iterator = SalaryIterator(salaries)

for salary in salary_iterator:
    print(salary)


Updated Salary: 75000.00
Updated Salary: 90000.00
Updated Salary: 82500.00


# Generators

In [12]:
#employee increment
def employee_increment() -> any:
    """Generator function that yields a 20% increment for each salary."""
    salaries = [100000, 200000, 150000]
    for salary in salaries:
        yield f"Increment received: {salary * 0.20:.2f}"


increments = employee_increment()

for increment in increments:
    print(increment)



Increment received: 20000.00
Increment received: 40000.00
Increment received: 30000.00


In [13]:
#Role-Based Salary generator

from typing import Generator, List


def salary_role(emp_roles: List[str]) -> Generator[str, None, None]:
    """
    Generator function that calculates new salaries for employees 
    based on their roles. If a role is not in the base salaries, it returns a message.
    """
    base_salaries = {
        "HR": 50000,
        "IT": 60000,
        "Finance": 70000,
        "Marketing": 55000
    }

    for role in emp_roles:
        if role in base_salaries:
            yield f"Role: {role}, New Salary: {base_salaries[role] * 1.20:.2f}"
        else:
            yield f"Role: {role}, No data available"


# Example usage
roles = ["IT", "HR", "Finance", "CEO"]
salary_generator = salary_role(roles)

for salary in salary_generator:
    print(salary)



Role: IT, New Salary: 72000.00
Role: HR, New Salary: 60000.00
Role: Finance, New Salary: 84000.00
Role: CEO, No data available


In [14]:
#infinite Iterator and generator:
import random
from typing import Generator


def random_employee_ids() -> Generator[int, None, None]:
    """
    Generator function that yields random employee IDs indefinitely.
    """
    while True:
        yield random.randint(100, 999)


# Example usage: Generate 5 random employee IDs
id_generator = iter(random_employee_ids())

for _ in range(5):
    print(next(id_generator))


377
432
773
174
787


In [15]:
# combinig all concpets to create a library management system
#book class
from typing import List, Generator


class Book:
    """Represents a book in the library."""

    lib_name = "SG Library"

    def __init__(self, b_id: int, title: str, author: str, copies: int) -> None:
        """Initializes the Book instance with ID, title, author, and copies available."""
        self.b_id = b_id
        self.title = title
        self.author = author
        self.copies = copies

    def show(self) -> None:
        """Displays book details."""
        print(f"ID: {self.b_id}, {self.title} by {self.author}, Copies: {self.copies}")

    @staticmethod
    def fine(days: int) -> int:
        """Calculates fine for overdue books (50 per day)."""
        return days * 50

    @classmethod
    def set_lib_name(cls, name: str) -> None:
        """Updates the library name."""
        cls.lib_name = name


class BookIterator:
    """Iterator for iterating through a list of books."""

    def __init__(self, books: List[Book]) -> None:
        """Initializes the BookIterator with a list of books."""
        self.books = books
        self.index = 0

    def __iter__(self) -> "BookIterator":
        """Returns the iterator object itself."""
        return self

    def __next__(self) -> Book:
        """Returns the next book in the list or raises StopIteration."""
        if self.index >= len(self.books):
            raise StopIteration
        book = self.books[self.index]
        self.index += 1
        return book


def book_availability(books: List[Book]) -> Generator[str, None, None]:
    """
    Generator function to check book availability.
    Yields available books with their copies or marks them as out of stock.
    """
    for book in books:
        if book.copies > 0:
            yield f"{book.title} - Available: {book.copies}"
        else:
            yield f"{book.title} - Out of Stock"


class Library:
    """Library management system to handle books."""

    def __init__(self, books: List[Book]) -> None:
        """Initializes the library with a list of books."""
        self.books = books

    def list_books(self) -> None:
        """Lists all books in the library."""
        print("\nBooks:")
        for book in BookIterator(self.books):
            book.show()

    def check_availability(self) -> None:
        """Checks availability of books."""
        print("\nAvailability:")
        for book in book_availability(self.books):
            print(book)

    def show_library_name(self) -> None:
        """Displays the library name."""
        print(f"\nLibrary: {Book.lib_name}")


books = [
    Book(1, "1984", "George Orwell", 4),
    Book(2, "Metamorphosis", "Franz Kafka", 2),
    Book(3, "Hippie", "Paulo Coelho", 0)
]

library = Library(books)

library.list_books()
library.check_availability()
library.show_library_name()

# Updating library name
Book.set_lib_name("Seg Library")
library.show_library_name()

# Calculating fine
print(f"{Book.fine(3)} rupees")






Books:
ID: 1, 1984 by George Orwell, Copies: 4
ID: 2, Metamorphosis by Franz Kafka, Copies: 2
ID: 3, Hippie by Paulo Coelho, Copies: 0

Availability:
1984 - Available: 4
Metamorphosis - Available: 2
Hippie - Out of Stock

Library: SG Library

Library: Seg Library
150 rupees
